# 🌱 Plant Disease Detection - Model Training

This notebook trains a custom CNN model for plant disease detection using:
- **MobileNetV2** (Transfer Learning)
- **PlantVillage Dataset** (87,000+ images, 38 classes)
- **TensorFlow.js Export** for browser deployment

---

## ⚡ Before Starting
1. Go to **Runtime > Change runtime type**
2. Select **GPU** as Hardware accelerator (T4 is fine)
3. Click **Save**

---

## Step 1: Install Dependencies

In [ ]:
!pip install tensorflow tensorflow-datasets tensorflowjs pillow -q
print("✅ Dependencies installed!")

## Step 2: Check GPU Availability

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus[0].name}")
    print(f"   TensorFlow version: {tf.__version__}")
else:
    print("❌ No GPU detected! Go to Runtime > Change runtime type > GPU")

## Step 3: Define Disease Classes (38 total)

In [ ]:
CLASS_LABELS = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",
    "Blueberry___healthy",
    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",
    "Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",
    "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Grape___healthy",
    "Orange___Haunglongbing_(Citrus_greening)",
    "Peach___Bacterial_spot",
    "Peach___healthy",
    "Pepper,_bell___Bacterial_spot",
    "Pepper,_bell___healthy",
    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",
    "Raspberry___healthy",
    "Soybean___healthy",
    "Squash___Powdery_mildew",
    "Strawberry___Leaf_scorch",
    "Strawberry___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites_Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy"
]

NUM_CLASSES = len(CLASS_LABELS)
IMG_SIZE = 224
BATCH_SIZE = 32

print(f"✅ {NUM_CLASSES} disease classes defined")

## Step 4: Load PlantVillage Dataset

This downloads ~1.2GB of plant leaf images (may take 5-10 minutes)

In [ ]:
import tensorflow_datasets as tfds

print("📥 Downloading PlantVillage dataset (this may take a few minutes)...")

# Load dataset
(train_ds, val_ds), info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

print(f"\n✅ Dataset loaded!")
print(f"   Total images: {info.splits['train'].num_examples:,}")
print(f"   Training: ~{int(info.splits['train'].num_examples * 0.8):,}")
print(f"   Validation: ~{int(info.splits['train'].num_examples * 0.2):,}")

## Step 5: Preprocess and Augment Data

In [ ]:
from tensorflow.keras import layers

# Data augmentation for training
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

def preprocess_train(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    image = data_augmentation(image, training=True)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def preprocess_val(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

# Apply preprocessing
train_ds = train_ds.map(preprocess_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(preprocess_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("✅ Data preprocessing configured!")

## Step 6: Build Model (MobileNetV2 + Custom Head)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

# Load pre-trained MobileNetV2
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze base model
base_model.trainable = False

# Build model
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✅ Model built!")
print(f"   Total params: {model.count_params():,}")
print(f"   Trainable params: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

## Step 7: Train - Phase 1 (Transfer Learning)

Training with frozen base model (takes ~15-20 minutes with GPU)

In [ ]:
print("🚀 Phase 1: Transfer Learning (frozen base)")
print("   This will take ~15-20 minutes...\n")

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2
    )
]

history1 = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=callbacks
)

print(f"\n✅ Phase 1 Complete!")
print(f"   Best validation accuracy: {max(history1.history['val_accuracy']):.2%}")

## Step 8: Train - Phase 2 (Fine-Tuning)

Unfreeze last 20 layers and train with lower learning rate

In [ ]:
print("🎯 Phase 2: Fine-tuning (last 20 layers)")
print("   This will take ~10-15 minutes...\n")

# Unfreeze base model
base_model.trainable = True

# Freeze all layers except last 20
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=callbacks
)

print(f"\n✅ Phase 2 Complete!")
print(f"   Final validation accuracy: {max(history2.history['val_accuracy']):.2%}")

## Step 9: Save Model

In [ ]:
import os

os.makedirs('saved_model', exist_ok=True)

# Save in Keras format
model.save('saved_model/plant_disease_model.keras')
print("✅ Model saved: saved_model/plant_disease_model.keras")

# Save in H5 format for TensorFlow.js conversion
model.save('saved_model/plant_disease_model.h5')
print("✅ H5 saved: saved_model/plant_disease_model.h5")

## Step 10: Export to TensorFlow.js

In [ ]:
import subprocess

os.makedirs('tfjs_model', exist_ok=True)

print("📦 Converting to TensorFlow.js format...")

!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_layers_model \
    saved_model/plant_disease_model.h5 \
    tfjs_model/

print("\n✅ TensorFlow.js model created!")
print("   Files in tfjs_model/:")
for f in os.listdir('tfjs_model'):
    size = os.path.getsize(f'tfjs_model/{f}')
    print(f"   - {f}: {size/1024:.1f} KB")

## Step 11: Create Labels JSON

In [ ]:
import json

labels_dict = {}
for i, label in enumerate(CLASS_LABELS):
    parts = label.split("___")
    crop = parts[0].replace("_", " ")
    condition = parts[1].replace("_", " ") if len(parts) > 1 else "Unknown"
    friendly_name = f"{crop} - {condition}".title()
    
    labels_dict[str(i)] = {
        "name": friendly_name,
        "crop": crop,
        "condition": condition
    }

with open('tfjs_model/labels.json', 'w') as f:
    json.dump(labels_dict, f, indent=2)

print("✅ Labels saved: tfjs_model/labels.json")
print(f"   {len(labels_dict)} classes")

## Step 12: Download Model Files

Download the `tfjs_model.zip` and extract to your project's `frontend/public/models/plant-disease/` folder

In [ ]:
import shutil
from google.colab import files

# Create zip file
shutil.make_archive('tfjs_model', 'zip', 'tfjs_model')

print("📥 Downloading tfjs_model.zip...")
print("\nAfter download:")
print("1. Extract the zip file")
print("2. Copy all files to: frontend/public/models/plant-disease/")
print("3. Run your app and select 'Custom Model' in Settings!")

files.download('tfjs_model.zip')

---

## 🎉 Training Complete!

### Next Steps:
1. Download `tfjs_model.zip` (should auto-download above)
2. Extract to `frontend/public/models/plant-disease/`
3. Run `npm install` in your project
4. Run `npm run dev`
5. Go to Settings → Select "Custom Model (Offline)"
6. Test with plant images!

### Model Performance:
- Expected accuracy: **~95%**
- Browser inference time: **<500ms**
- Model size: **~15MB**